In [92]:
import pandas as pd
import numpy as np

In [93]:
features = pd.read_csv('team_features.csv')
matches = pd.read_csv('MNCAATourneyCompactResults.csv')

In [94]:
matches.drop(columns = ['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'])

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325
...,...,...,...
2513,2024,1301,1181
2514,2024,1345,1397
2515,2024,1163,1104
2516,2024,1345,1301


In [95]:
print(matches.columns)
print(features.columns)

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT'],
      dtype='object')
Index(['index', 'TeamID', 'Season', 'seed_num', 'weighted_past_seed',
       '3pt_success_rate', 'freethrow_success_rate', 'field_success_rate'],
      dtype='object')


In [96]:
matches = pd.merge(matches[['Season', 'WTeamID', 'LTeamID']], features[['TeamID', 'Season', 'weighted_past_seed',
       '3pt_success_rate', 'freethrow_success_rate', 'field_success_rate']], left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

matches = matches.drop(columns = ['TeamID'])
matches = matches.rename(columns = {'weighted_past_seed': 'weighted_past_seed_W', 
                                    '3pt_success_rate': '3pt_success_rate_W', 
                                    'freethrow_success_rate': 'freethrow_success_rate_W',
                                    'freethrow_success_rate': 'freethrow_success_rate_W',
                                    'field_success_rate': 'field_success_rate_W'
                                    })

In [97]:
matches.columns

Index(['Season', 'WTeamID', 'LTeamID', 'weighted_past_seed_W',
       '3pt_success_rate_W', 'freethrow_success_rate_W',
       'field_success_rate_W'],
      dtype='object')

In [98]:
matches = pd.merge(matches[['Season', 'WTeamID', 'LTeamID', 'weighted_past_seed_W',
       '3pt_success_rate_W', 'freethrow_success_rate_W',
       'field_success_rate_W']], features[['TeamID', 'Season', 'weighted_past_seed',
       '3pt_success_rate', 'freethrow_success_rate', 'field_success_rate']], left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

matches = matches.drop(columns=['TeamID'])
matches = matches.rename(columns = {'weighted_past_seed': 'weighted_past_seed_L', 
                                    '3pt_success_rate': '3pt_success_rate_L', 
                                    'freethrow_success_rate': 'freethrow_success_rate_L',
                                    'field_success_rate': 'field_success_rate_L'
                                    })

In [99]:
all_cols = {'weighted_past_seed', '3pt_success_rate', 'freethrow_success_rate', 'field_success_rate'}
for col in all_cols:
    matches[col+'_diff'] = matches[col+'_W'] - matches[col+'_L']


In [100]:
matches = matches[[col+'_diff' for col in all_cols]].copy()  #must do copy to create a new df, not a slice
matches_negated = matches.copy()
for col in all_cols:
    matches_negated[col+'_diff'] = - matches[col+'_diff']

matches['label'] = np.full(len(matches), 1) #diff features have true label 1
matches_negated['label'] = np.full(len(matches_negated), 0)

In [101]:
data = pd.concat([matches, matches_negated])
data.columns

Index(['weighted_past_seed_diff', '3pt_success_rate_diff',
       'freethrow_success_rate_diff', 'field_success_rate_diff', 'label'],
      dtype='object')

In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# # Concatenate and prepare the data
# X = data.drop(columns='label').values.astype('float32')
# Y = data['label'].values.astype('float32')

# # Standardize features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Split the dataset
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# # Convert to PyTorch tensors
# X_train_tensor = torch.tensor(X_train)
# Y_train_tensor = torch.tensor(Y_train).unsqueeze(1)
# X_test_tensor = torch.tensor(X_test)
# Y_test_tensor = torch.tensor(Y_test).unsqueeze(1)

# # Create DataLoaders
# train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # Define a simple neural network
# class Classifier(nn.Module):
#     def __init__(self, input_dim):
#         super(Classifier, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, 1),
#             nn.Sigmoid()
#         )
        
#     def forward(self, x):
#         return self.net(x)

# # Initialize model, loss function, and optimizer
# model = Classifier(input_dim=X.shape[1])
# criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# epochs = 100
# for epoch in range(epochs):
#     model.train()
#     epoch_loss = 0
#     for X_batch, Y_batch in train_loader:
#         optimizer.zero_grad()
#         output = model(X_batch)
#         loss = criterion(output, Y_batch)
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# # Evaluation on test data
# model.eval()
# with torch.no_grad():
#     predictions = model(X_test_tensor)
#     predicted_labels = (predictions >= 0.5).float()
#     accuracy = (predicted_labels == Y_test_tensor).float().mean()
#     print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/100, Loss: 77.4075
Epoch 2/100, Loss: 74.4650
Epoch 3/100, Loss: 74.3069
Epoch 4/100, Loss: 74.0622
Epoch 5/100, Loss: 74.0348
Epoch 6/100, Loss: 73.9467
Epoch 7/100, Loss: 73.8816
Epoch 8/100, Loss: 73.9873
Epoch 9/100, Loss: 73.8842
Epoch 10/100, Loss: 73.8214
Epoch 11/100, Loss: 73.7184
Epoch 12/100, Loss: 73.6407
Epoch 13/100, Loss: 73.6477
Epoch 14/100, Loss: 73.5365
Epoch 15/100, Loss: 73.6837
Epoch 16/100, Loss: 73.5668
Epoch 17/100, Loss: 73.5495
Epoch 18/100, Loss: 73.4951
Epoch 19/100, Loss: 73.4366
Epoch 20/100, Loss: 73.3543
Epoch 21/100, Loss: 73.3790
Epoch 22/100, Loss: 73.3572
Epoch 23/100, Loss: 73.2105
Epoch 24/100, Loss: 73.3361
Epoch 25/100, Loss: 73.1483
Epoch 26/100, Loss: 73.2563
Epoch 27/100, Loss: 73.2565
Epoch 28/100, Loss: 73.0386
Epoch 29/100, Loss: 73.0546
Epoch 30/100, Loss: 72.8909
Epoch 31/100, Loss: 73.0344
Epoch 32/100, Loss: 72.9648
Epoch 33/100, Loss: 72.9608
Epoch 34/100, Loss: 72.9499
Epoch 35/100, Loss: 73.0846
Epoch 36/100, Loss: 72.7797
E

In [108]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = data.drop(columns='label').values.astype('float32')
Y = data['label'].values.astype('float32')
X = scaler.fit_transform(X)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.8, random_state=42)

In [109]:
X_train_tensor = torch.tensor(X_train)
X_val_tensor = torch.tensor(X_val)
Y_train_tensor = torch.tensor(Y_train).unsqueeze(1)
Y_val_tensor = torch.tensor(Y_val).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [132]:
from torch import nn
class FCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid())       #nn.Sequential takes in batchsize x input_dim
    def forward(self, x):
        return self.layers(x)

In [133]:
model = FCNN(X_train.shape[1])
criterion = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr = 0.01)
#learning rate too high will cause exploding weights -> NaNs in y_pred

In [134]:
model.eval()
max_epoch = 200
for epoch in range(max_epoch):
    avg_loss = 0
    for x, y in train_loader:
        y_pred = model(x)
        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()

        avg_loss += loss.item()/len(train_loader)
    if (epoch + 1) % 10 == 0:
        print(f"epoch {epoch+1}: Loss {avg_loss: .4f}")

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in val_loader:
        y_pred = model(x)
        pred = y_pred >= 0.5
        correct += (pred == y).sum().item()
        total += len(x)
print(f"validation accuracy: {correct/total: .2%}")

epoch 10: Loss  0.5882
epoch 20: Loss  0.5854
epoch 30: Loss  0.5850
epoch 40: Loss  0.5840
epoch 50: Loss  0.5839
epoch 60: Loss  0.5828
epoch 70: Loss  0.5827
epoch 80: Loss  0.5828
epoch 90: Loss  0.5820
epoch 100: Loss  0.5816
epoch 110: Loss  0.5809
epoch 120: Loss  0.5806
epoch 130: Loss  0.5803
epoch 140: Loss  0.5806
epoch 150: Loss  0.5784
epoch 160: Loss  0.5788
epoch 170: Loss  0.5789
epoch 180: Loss  0.5774
epoch 190: Loss  0.5779
epoch 200: Loss  0.5766
validation accuracy:  68.25%


In [135]:
X_train.shape[1]

4